Sumedha Chalasani

Risk Hazard System to Assess Probability of Violating HIPAA

In [ ]:
import pandas as pd

fhandle = open('healthcare_dataset.csv','r')
fdata = pd.read_csv('healthcare_dataset.csv')

#fdata.shape
#fdata.isnull()
#fdata.dtypes
fdata.head()

In [ ]:
import seaborn as sns
sns.pairplot(
    fdata[["Age", "Billing Amount"]],
    height = 3)

In [ ]:
import numpy as np
bill = []
age = []
for i in fdata['Billing Amount']:
    bill.append(i)
for j in fdata['Age']:
    age.append(j)
myarr = np.empty((6932,2))
myarr[:,0] = age
myarr[:,1] = bill

xvar = myarr[:,0]
xvar = xvar.reshape(-1,1)
yvar = myarr[:,1]
yvar = yvar.reshape(-1,1)

from sklearn.model_selection import train_test_split
xvar_train, xvar_test, yvar_train, yvar_test = train_test_split(xvar, yvar, train_size = 0.60, random_state = 1)
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(xvar_train, yvar_train)

from sklearn.preprocessing import StandardScaler
std = StandardScaler()
afp = np.append(xvar_train, xvar_test)
afp = afp.reshape(-1,1)
std.fit(afp)

testscore = lr.score(xvar_test, yvar_test)
trainscore = lr.score(xvar_train, yvar_train)

y_pred = lr.predict(xvar_test)
from sklearn.metrics import mean_squared_error
import math
error = math.sqrt(mean_squared_error(yvar_test, y_pred))
modelval = lr.predict(xvar_train[:1])
actualval = yvar_train[:1]

print(f'The training data score is {trainscore} while the testing data score is {testscore}. The square root of the mean squared error is {error}, meaning the model predicts with a standard deviation of {error}. Therefore, when predicting the first row, the predicted value is {modelval} while the actual first row value is {actualval}.')
print("\nThe coefficient of determination for both the training and testing data is low because age and billing amount don't have any correlation in this particular dataset. However, this linear regression model can be used with other datasets to predict billing amount based on age, which can help determine the risk hazard for violating HIPAA.")

In [ ]:
fhandle1 = open('samplehealthplan.txt','r')
insurancelist = []
lines = fhandle1.readlines()
for line in fdata['Insurance Provider']:
    if not line in insurancelist:
        insurancelist.append(line)   

HIPAA_insurance = []
for line in lines:
    if line.find("not covered under HIPAA") != -1 or line.find("does not pay the cost of healthcare") != -1:
        val = 1
    else:
        val = 0

if val == 1:
    for i in insurancelist:
        for line in lines:
            if i in line and not i in HIPAA_insurance:
                HIPAA_insurance.append(i)
elif val == 0:
    for i in insurancelist:
        for line in lines:
            if i in line and not i+'(NA)' in HIPAA_insurance:
                HIPAA_insurance.append(i+'(NA)')

medicarerisk = 0
count = 0
for j in fdata['Insurance Provider']:
    count += 1
    if j == "Medicare":
        ageval = fdata.iloc[count,1]
        if ageval > 45:
            medicarerisk += 5
        else:
            medicarerisk += 1
        billval = fdata.iloc[count, 9]
        if billval > 30000:
            medicarerisk += 5
        else:
            medicarerisk += 1
if "Medicare(NA)" in HIPAA_insurance:
    medicarerisk *= 0.5
elif "Medicare" in HIPAA_insurance:
    medicarerisk *= 2
medicarerisk /= 6932
print(f'The risk rating for Medicare is {medicarerisk}, calculated with HIPAA applicability, age, and billing amount taken into account.')

if medicarerisk > 0.50:
    print('The risk rating is higher than average, so transactions and operations regarding patients with Medicare should be monitored carefully in order reduce the probability of violating HIPAA.')

Sources:
https://www.hhs.gov/sites/default/files/ocr/privacy/hipaa/administrative/combined/hipaa-simplification-201303.pdf
(list of hipaa guidelines, looking at definition of health care plans specifically (pg 15))

https://www.kaggle.com/datasets/prasad22/healthcare-dataset
(kaggle dataset)